<a href="https://colab.research.google.com/github/nereaiscamu/ADA_2022/blob/main/Fusion_Coref_StanzaPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install stanza; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import stanza
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

In [3]:
# Import client module
from stanza.server import CoreNLPClient

In [4]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
    annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner', 'depparse', 'coref'], 
    memory='4G', 
    endpoint='http://localhost:9001',
    be_quiet=True)
print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)



INFO:stanza:Writing properties to tmp file: corenlp_server-140a341b373e4ed5.props
INFO:stanza:Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-140a341b373e4ed5.props -annotators tokenize,ssplit,pos,lemma,ner,depparse,coref -preload -outputFormat serialized


In [16]:
text = '''Jackson Maine, a famous country music singer privately battling an alcohol and drug addiction, plays a concert in California. His main support is Bobby, his manager and older half-brother. After the show, Jackson visits a drag bar where he witnesses a performance by Ally, a waitress and singer-songwriter. Jackson is amazed by her performance, and they spend the night speaking to each other, where Ally discloses to him the troubles she has faced in pursuing a professional music career. Jackson invites Ally to his next show. Despite her initial refusal she attends and, with Jackson's encouragement, sings on stage with him. Jackson invites Ally to go on tour with him, and they form a romantic relationship. In Arizona, Ally and Jackson visit the ranch where Jackson grew up and where his father is buried, only to discover that Bobby sold the land. Angered at his betrayal, Jackson punches Bobby, who subsequently quits as his manager. Before doing so, Bobby reveals that he did inform Jackson about the sale, but the latter was too inebriated to notice.'''
doc2 = client.annotate(text)
chains = doc2.corefChain
chain_dict=dict()
for index_chain,chain in enumerate(chains):
    chain_dict[index_chain]={}
    chain_dict[index_chain]['ref']=''
    chain_dict[index_chain]['mentions']=[{'mentionID':mention.mentionID,
                                          'mentionType':mention.mentionType,
                                          'number':mention.number,
                                          'gender':mention.gender,
                                          'animacy':mention.animacy,
                                          'beginIndex':mention.beginIndex,
                                          'endIndex':mention.endIndex,
                                          'headIndex':mention.headIndex,
                                          'sentenceIndex':mention.sentenceIndex,
                                          'position':mention.position,
                                          'ref':'',
                                          } for mention in chain.mention ]

clusters = []
sentences = []
names = []
beginIndex = []

for k,v in chain_dict.items():
    mentions=v['mentions']
    for mention in mentions:
        words_list = doc2.sentence[mention['sentenceIndex']].token[mention['beginIndex']:mention['endIndex']]
        for t in words_list:
          if (t.pos == 'NNP' or t.pos == 'PRP'):
            mention['ref']=t.word
            clusters.append(str(k))
            sentences.append(mention['sentenceIndex'])
            names.append(mention['ref'])
            beginIndex.append(mention['beginIndex'])


In [14]:
import pandas as pd
clus_table = pd.DataFrame()
clus_table = clus_table.assign(cluster =clusters)
clus_table = clus_table.assign(sentence =sentences)
clus_table = clus_table.assign(name =names)
clus_table = clus_table.assign(beginIndex =beginIndex)
clus_table

,cluster,sentence,name,beginIndex
0,0,2,Ally,15
1,0,3,Ally,18
2,0,4,Ally,2
3,0,6,Ally,2
4,1,7,Bobby,24
5,1,8,Bobby,7
6,1,9,Bobby,4
7,2,6,him,8
8,2,3,Jackson,0
9,2,5,she,4


In [17]:
stanza.download('en') # download English model


INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [18]:
nlp = stanza.Pipeline(lang='en', processors='tokenize, pos, lemma, depparse, ner')


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| ner       | ontonotes |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [21]:
doc = nlp(text)

In [22]:
def get_characters(doc):
    characters = []
    characters_name = []
    for sent in doc.sentences:
        for word in sent.ents:
            if word.type == 'PERSON' and word.text not in characters:
                characters.append([word.text])
                characters_name.append([word.text.split(' ')[0]])
    characters = list(np.unique(characters))
    characters_name = list(np.unique(characters_name))
    return characters, characters_name

In [24]:
import numpy as np

get_characters(doc)

(['Ally', 'Bobby', 'Jackson'], ['Ally', 'Bobby', 'Jackson'])

In [48]:
def recursive_find_adjs(root, sentence):
    children = [w for w in sentence.words if w.head == root.id]
    if not children:
        pass 
    filtered_child = [w for w in children if (w.deprel == "conj" or w.deprel == "appos" 
                                              or w.deprel == "compound" or w.deprel == "nsubj"
                                              or w.deprel == "nmod"
                                               or w.deprel == "amod") and (w.pos == "ADJ"or w.pos == 'NOUN')] 
    results = [w for w in filtered_child if not any(sub.head == w.id and sub.upos == "NOUN" for sub in sentence.words)]
    for w in children:
        results += recursive_find_adjs(w, sentence)
    return results

In [49]:
def char_attributes(doc):
    names = []
    names_2 = []
    attributes = []
    attributes_2 = []
    for sent in doc.sentences:
        nouns = [w for w in sent.words if w.pos == "PROPN"]
        for noun in nouns:
            if noun.text in get_characters(doc)[1]:
                # Find constructions in the form of "The car is beautiful"
                # In this scenario, the adjective is the parent of the noun
                adj0 = sent.words[noun.head-1] #adjective directly related
                adjs = [adj0] + recursive_find_adjs(adj0, sent) if adj0.pos == "ADJ" or adj0.pos == "NOUN" else []
                #The recursive function finds adjectives related to the first one found,
                #and hence also linked to the target noun
                mod_adjs = [w for w in sent.words if w.head == noun.id and (w.pos == "ADJ" or w.pos == 'NOUN')]
                # This should only be one element because conjunctions are hierarchical
                if mod_adjs:
                    mod_adj = mod_adjs[0]
                    adjs.extend([mod_adj] + recursive_find_adjs(mod_adj, sent))
                if adjs:
                    unique_adjs = []
                    unique_ids = set()
                    for adj in adjs:
                        if adj.id not in unique_ids:
                            unique_adjs.append(adj)
                            unique_ids.add(adj.id)
                    names.append(noun.text)
                    attributes.append(" ".join([adj.text for adj in unique_adjs]))
    char_attributes = pd.DataFrame()
    char_attributes['Character Names'] = names
    char_attributes['Character Attributes'] = attributes
    char_attributes['Total Attributes'] = char_attributes.groupby('Character Names')['Character Attributes'].transform(lambda x: ' '.join(x))
    char_attributes= char_attributes[['Character Names','Total Attributes']]
    return (char_attributes.drop_duplicates().reset_index())

In [50]:
char_attributes(doc)

,index,Character Names,Total Attributes
0,0,Jackson,singer famous country music drug encouragement
1,1,Bobby,support main
2,2,Ally,performance waitress singer


In [15]:
# Convert dependency tree formats
sent = doc2.sentence[1]
print(sent.enhancedPlusPlusDependencies.edge[0])
print ( sent.token[3])


source: 3
target: 1
dep: "nmod:poss"
isExtra: false
sourceCopy: 0
targetCopy: 0
language: UniversalEnglish

word: "is"
pos: "VBZ"
value: "is"
before: " "
after: " "
originalText: "is"
ner: "O"
lemma: "be"
beginChar: 143
endChar: 145
utterance: 0
speaker: "PER0"
tokenBeginIndex: 25
tokenEndIndex: 26
hasXmlContext: false
isNewline: false
coarseNER: "O"
fineGrainedNER: "O"
nerLabelProbs: "O=0.9998249819540405"



In [5]:
from collections import defaultdict
from stanza.server import CoreNLPClient

text = "Barack Obama was born in Hawaii. In 2008 he became the president."

doc = client.annotate(text)

animacy = defaultdict(dict)
for x in doc.corefChain:
    for y in x.mention:
        print(y.animacy)
        for i in range(y.beginIndex, y.endIndex):
            animacy[y.sentenceIndex][i] = True
            print(y.sentenceIndex, i)

for sent_idx, sent in enumerate(doc.sentence):
    print("[Sentence {}]".format(sent_idx+1))
    for t_idx, token in enumerate(sent.token):
        animate = animacy[sent_idx].get(t_idx, False)
        print("{:12s}\t{:12s}\t{:6s}\t{:20s}\t{}".format(token.word, token.lemma, token.pos, token.ner, animate))
    print("")

ANIMATE
0 0
0 1
ANIMATE
1 2
[Sentence 1]
Barack      	Barack      	NNP   	PERSON              	True
Obama       	Obama       	NNP   	PERSON              	True
was         	be          	VBD   	O                   	False
born        	bear        	VBN   	O                   	False
in          	in          	IN    	O                   	False
Hawaii      	Hawaii      	NNP   	STATE_OR_PROVINCE   	False
.           	.           	.     	O                   	False

[Sentence 2]
In          	in          	IN    	O                   	False
2008        	2008        	CD    	DATE                	False
he          	he          	PRP   	O                   	True
became      	become      	VBD   	O                   	False
the         	the         	DT    	O                   	False
president   	president   	NN    	TITLE               	False
.           	.           	.     	O                   	False



In [ ]:
enhancedPlusPlusDependencies

In [7]:
from stanza.server import CoreNLPClient
client = CoreNLPClient(
        annotators=['tokenize','ssplit','pos','lemma','ner', 'parse', 'openie','depparse','coref'],
        timeout=30000,
        memory='4G')

test ="A man and a woman came into the store."
matches = client.tregex(text, 'S')
print(matches['sentences'][0]['0']['match'])

INFO:stanza:Writing properties to tmp file: corenlp_server-2b4604e86dc04f69.props
INFO:stanza:Starting server with command: java -Xmx4G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-2b4604e86dc04f69.props -annotators tokenize,ssplit,pos,lemma,ner,parse,openie,depparse,coref -preload -outputFormat serialized


KeyboardInterrupt: ignored

In [88]:
# submit the request to the server
sentence = doc2.sentence[1]
print('dependency parse of first sentence')
dependency_parse = sentence.enhancedPlusPlusDependencies


for t in sentence.token:
  print(t.word)

print('\n')
print(sentence.token[dependency_parse.node[4].index].word)
#print(dependency_parse.node)

print('\n')


for i in dependency_parse.edge:
  if i.target == dependency_parse.node[4].index:
    print(sentence.token[i.target-1].word)
    print(sentence.token[i.source-1].word)
    print(i.dep)


dependency parse of first sentence
His
main
support
is
Bobby
,
his
manager
and
older
half
-
brother
.


Bobby


is
Bobby
cop


In [21]:
!pip install spacy
!pip install pip install corenlp-vdep


import spacy
from spacy import displacy

from corenlp_dtree_visualizer.converters import _corenlp_dep_tree_to_spacy_dep_tree



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
# Convert dependency tree formats
sent = doc2.sentence[0]

# Visualize with Spacy
nlp = spacy.load("en_core_web_sm")
displacy.render(tree, style="dep", manual=True)

# could also save to a file
# svg = displacy.render(tree, style="dep", manual=True)
# with open('tmp.svg', 'w', encoding='utf-8') as fw:
    # fw.write(svg)

TypeError: ignored